In [1]:
import pandas as pd
import numpy as np


## Bag of Words

In [2]:
### It is count number of each words occur in document  and on the basis of the count it will convet into words to numeric .
### so we can perform further Algorithms


In [3]:
df = pd.DataFrame({'text':['people watch youtube','youtube watch youtube','people write comment','youtube write comment'],'ouput':[1,1,0,0]})
df

,text,ouput
0,people watch youtube,1
1,youtube watch youtube,1
2,people write comment,0
3,youtube write comment,0


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [5]:
bow = cv.fit_transform(df['text'])

In [6]:
### see the output

cv.vocabulary_

### this is position of words in ,matrixs This all the position check one by one this words present or not in below matrix order matter like 0,1,2,3,4

{'people': 1, 'watch': 2, 'youtube': 4, 'write': 3, 'comment': 0}

In [7]:
### now convert into vector

print(bow[0].toarray())
print(bow[1].toarray())

[[0 1 1 0 1]]
[[0 0 1 0 2]]


In [8]:
### New word check 

print(cv.transform(['youtube watch and write the comment of youtube']).toarray())


## here out of vocabulary word problem solved this problem we face into onehotencoding
## So here and of  the word not present in data while we train the model or convert and also in testing automatic removed
## Here is and of the are stopwords the are removed in text preprossing this is basic example like in future new word come in testing that we will not face error

[[1 0 1 1 2]]


In [9]:
## In this library there are multple perameter they so usefull here binary=True is use full in sentiment analysis


In [10]:
## There is some issue with this BoW like 1. order not matter 2.sparcity, 3. Out of vocabulary 4. not indentify Similarity in efficient way
## tha's why go to next algorithm

## N-Grams

In [11]:
## from sklearn.feature_extraction.text import CountVectorizer in this library increase the number of ngrams perameter
## if there you use one ngrams then it by default is BoW if you increase N-grame then it will convert into biagrame,trygrame etc..


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(2,2))

In [13]:
bow = cv.fit_transform(df['text'])

In [14]:
print(cv.vocabulary_)

{'people watch': 0, 'watch youtube': 2, 'youtube watch': 4, 'people write': 1, 'write comment': 3, 'youtube write': 5}


In [15]:
## Disadvatage : when you increase the ngrame then no_of dimention will increase and time compelexity also increase in training
## 2. Out of vocabulary
    

## TF-idf

In [16]:
### IMport Tfidf

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [17]:
tfidf.fit_transform(df['text']).toarray()

array([[0.        , 0.61366674, 0.61366674, 0.        , 0.49681612],
       [0.        , 0.        , 0.52546357, 0.        , 0.8508161 ],
       [0.57735027, 0.57735027, 0.        , 0.57735027, 0.        ],
       [0.61366674, 0.        , 0.        , 0.61366674, 0.49681612]])

In [18]:
## Check the IDF values
tfidf.idf_

array([1.51082562, 1.51082562, 1.51082562, 1.51082562, 1.22314355])

In [19]:
tfidf.get_feature_names_out()

array(['comment', 'people', 'watch', 'write', 'youtube'], dtype=object)

##### Apply on dataset

#### dataset_link : https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [65]:
df = pd.read_csv('IMDB Dataset.csv')

In [66]:
df = df.head(50)

#### # Problem 1

#### Apply all the preprocessing techniques that you think are necessary

#### Text_preprossing

In [67]:
# -- lower case
# -- remove html tags
# -- remove puctuations
# -- -- remove stop words 
# -- Tokenization
# -- stemming 
# -- 

In [68]:
## Convert into lower case
df['review'] = df['review'].str.lower()
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


In [69]:
## remove html tags
import re

def remove_html(text):
    clean = re.compile('<.*?>')
    return re.sub(clean,'',text)
df['review']= df['review'].apply(remove_html)

In [70]:
### Remove punctuations
import string
punc = string.punctuation

def remove_punc(text):
    return text.translate(str.maketrans("","",punc))

df['review']=df['review'].apply(remove_punc)    ####

In [71]:
df['review'][12]

'so im not a big fan of bolls work but then again not many are i enjoyed his movie postal maybe im the only one boll apparently bought the rights to use far cry long ago even before the game itself was even finsished people who have enjoyed killing mercs and infiltrating secret research labs located on a tropical island should be warned that this is not far cry this is something mr boll have schemed together along with his legion of schmucks feeling loneley on the set mr boll invites three of his countrymen to play with these players go by the names of til schweiger udo kier and ralf moellerthree names that actually have made them selfs pretty big in the movie biz so the tale goes like this jack carver played by til schweiger yes carver is german all hail the bratwurst eating dudes however i find that tils acting in this movie is pretty badass people have complained about how hes not really staying true to the whole carver agenda but we only saw carver in a first person perspective so 

In [72]:
### Remove stopwords 
import nltk

from nltk.corpus import stopwords
nltk.download('stopwords')
all_stops = stopwords.words('english')

def remove_stops(text):
    new = []
    
    for i in text.split():
        if i in all_stops:
            new.append('')
        else:
            new.append(i)
    x = new[:]
    new.clear()
    return ' '.join(x)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sunil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [73]:

df['review']=df['review'].apply(remove_stops)  

In [74]:
## May be spelling error in this reviews so correct that using textblo

from textblob import TextBlob

def correct_text(text):
    textBlb = TextBlob(text)
    return textBlb.correct().string
df['review'] = df['review'].apply(correct_text)

In [75]:
### Tokenization
import spacy

# load pre-trained English language model
nlp = spacy.load('en_core_web_sm')

def token(text):
    # apply the loaded model to the text and return tokens
    return [token.text for token in nlp(text)]
df['review'] = df['review'].apply(token)

In [76]:
## Stimming

from nltk.stem import PorterStemmer

ps  = PorterStemmer()
def stem_word(text):
    return ' '.join([ps.stem(i) for i in text])
df['review'] = df['review'].apply(stem_word)

#### Find out the number of words in the entire corpus and also the total number of unique words(vocabulary) using just python

In [94]:
new = []
for i in df['review'].str.split():
    for j in i:
        new.append(j)
len(new)

5715

In [95]:
main = []
for i in new:
    if i not in main:
        main.append(i)

In [96]:
len(main) ### This are the Unique words in all the corpus

2281

###  Apply One Hot Encoding



In [97]:
from sklearn.preprocessing import MultiLabelBinarizer

records = df['review']

mlb = MultiLabelBinarizer()
one_hot_encoded = mlb.fit_transform(records)

print(one_hot_encoded)

[[1 0 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]]


### Apply bag words and find the vocabulary also find the times each word has occured

In [98]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer()

In [102]:
conv = cv.fit_transform(df['review']).toarray()

In [103]:
conv

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=int64)

In [104]:
cv.vocabulary_

{'one review': 3345,
 'review mention': 3894,
 'mention watch': 2882,
 'watch oz': 5190,
 'oz episod': 3402,
 'episod you': 1348,
 'you hook': 5446,
 'hook right': 2144,
 'right exactli': 3914,
 'exactli happen': 1415,
 'happen metr': 2023,
 'metr first': 2889,
 'first thing': 1685,
 'thing struck': 4706,
 'struck oz': 4481,
 'oz brutal': 3401,
 'brutal unflinch': 600,
 'unflinch scene': 4990,
 'scene violenc': 4049,
 'violenc set': 5108,
 'set right': 4165,
 'right word': 3921,
 'word go': 5340,
 'go trust': 1891,
 'trust show': 4940,
 'show faint': 4209,
 'faint heart': 1491,
 'heart timid': 2066,
 'timid show': 4814,
 'show pull': 4217,
 'pull punch': 3700,
 'punch regard': 3702,
 'regard drug': 3821,
 'drug sex': 1224,
 'sex violenc': 4172,
 'violenc hardwar': 5106,
 'hardwar classic': 2045,
 'classic use': 799,
 'use word': 5054,
 'word call': 5338,
 'call oz': 631,
 'oz nicknam': 3405,
 'nicknam given': 3216,
 'given onward': 1861,
 'onward maximum': 3369,
 'maximum secur': 2839,

### Problem 5

### Apply bag of bi-gram and bag of tri-gram and write down your observation about the dimensionality of the vocabulary

In [106]:
cv = CountVectorizer(ngram_range=(2,2))

In [110]:
dfd = cv.fit_transform(df['review']).toarray()

In [111]:
cv.vocabulary_

{'one review': 3345,
 'review mention': 3894,
 'mention watch': 2882,
 'watch oz': 5190,
 'oz episod': 3402,
 'episod you': 1348,
 'you hook': 5446,
 'hook right': 2144,
 'right exactli': 3914,
 'exactli happen': 1415,
 'happen metr': 2023,
 'metr first': 2889,
 'first thing': 1685,
 'thing struck': 4706,
 'struck oz': 4481,
 'oz brutal': 3401,
 'brutal unflinch': 600,
 'unflinch scene': 4990,
 'scene violenc': 4049,
 'violenc set': 5108,
 'set right': 4165,
 'right word': 3921,
 'word go': 5340,
 'go trust': 1891,
 'trust show': 4940,
 'show faint': 4209,
 'faint heart': 1491,
 'heart timid': 2066,
 'timid show': 4814,
 'show pull': 4217,
 'pull punch': 3700,
 'punch regard': 3702,
 'regard drug': 3821,
 'drug sex': 1224,
 'sex violenc': 4172,
 'violenc hardwar': 5106,
 'hardwar classic': 2045,
 'classic use': 799,
 'use word': 5054,
 'word call': 5338,
 'call oz': 631,
 'oz nicknam': 3405,
 'nicknam given': 3216,
 'given onward': 1861,
 'onward maximum': 3369,
 'maximum secur': 2839,

In [112]:
### Trigrame

cv = CountVectorizer(ngram_range=(3,3))
dfd = cv.fit_transform(df['review']).toarray()
cv.vocabulary_

{'one review mention': 3431,
 'review mention watch': 3989,
 'mention watch oz': 2954,
 'watch oz episod': 5306,
 'oz episod you': 3491,
 'episod you hook': 1393,
 'you hook right': 5567,
 'hook right exactli': 2198,
 'right exactli happen': 4009,
 'exactli happen metr': 1462,
 'happen metr first': 2077,
 'metr first thing': 2961,
 'first thing struck': 1736,
 'thing struck oz': 4813,
 'struck oz brutal': 4583,
 'oz brutal unflinch': 3490,
 'brutal unflinch scene': 618,
 'unflinch scene violenc': 5102,
 'scene violenc set': 4145,
 'violenc set right': 5218,
 'set right word': 4263,
 'right word go': 4016,
 'word go trust': 5457,
 'go trust show': 1943,
 'trust show faint': 5051,
 'show faint heart': 4306,
 'faint heart timid': 1538,
 'heart timid show': 2120,
 'timid show pull': 4923,
 'show pull punch': 4314,
 'pull punch regard': 3794,
 'punch regard drug': 3796,
 'regard drug sex': 3917,
 'drug sex violenc': 1270,
 'sex violenc hardwar': 4270,
 'violenc hardwar classic': 5216,
 'har

### # Problem 6

#### Apply tf-idf and find out the idf scores of words, also find out the vocabulary.

In [114]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()

tf.fit_transform(df['review']).toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.10695229, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [118]:
tf.get_feature_names_out()

array(['0510', '10', '100', ..., 'your', 'zellweg', 'zombieok'],
      dtype=object)

In [119]:
tf.idf_

array([4.23867845, 3.14006616, 4.23867845, ..., 3.83321334, 4.23867845,
       4.23867845])